In [1]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
progress = ProgressBar().register()

In [4]:
IMG_DIR = '../../analysis'
DPI=120

## Loading data using Dask (loads lazily)
* https://www.youtube.com/watch?v=RA_2qdipVng&t=1s
* http://matthewrocklin.com/slides/scipy-2017.html

In [20]:
mkdir data
cd data

In [ ]:
# http://stat-computing.org/dataexpo/2009/the-data.html
# !curl -O http://stat-computing.org/dataexpo/2009/2000.csv.bz2
# !curl -O http://stat-computing.org/dataexpo/2009/2001.csv.bz2
# !curl -O http://stat-computing.org/dataexpo/2009/2002.csv.bz2
# !ls -lh

In [26]:
# !bzip2 -d 2000.csv.bz2
# !bzip2 -d 2001.csv.bz2
# !bzip2 -d 2002.csv.bz2
# !ls -lh

In [31]:
# data_types = {'CRSElapsedTime': int, 'CRSDepTime': int, 'Year': int, 'Month': int, 'DayOfWeek': int, 'DayofMonth': int}
data_types = {'CRSDepTime': int, 'Year': int, 'Month': int, 'DayOfWeek': int, 'DayofMonth': int}


# http://dask.pydata.org/en/latest/dataframe-overview.html
%time df = dd.read_csv('./*.csv', encoding='iso-8859-1', dtype=data_types, assume_missing=True)

CPU times: user 132 ms, sys: 8 ms, total: 140 ms
Wall time: 223 ms


In [32]:
# %time df.head()

[########################################] | 100% Completed |  1.6s
CPU times: user 1.45 s, sys: 144 ms, total: 1.6 s
Wall time: 1.65 s


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2000,1,28,5,1647.0,1647,1906.0,1859.0,HP,154.0,...,15.0,11.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2000,1,29,6,1648.0,1647,1939.0,1859.0,HP,154.0,...,5.0,47.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2000,1,30,7,NaN,1647,NaN,1859.0,HP,154.0,...,0.0,0.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2000,1,31,1,1645.0,1647,1852.0,1859.0,HP,154.0,...,7.0,14.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2000,1,1,6,842.0,846,1057.0,1101.0,HP,609.0,...,3.0,8.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [33]:
%time df = df.fillna(-1)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.85 ms


In [8]:
# Takes a while
# %time df.count().compute()

In [9]:
# Takes a while, but should be doable
# %time unique_origins = df['Origin'].unique().compute()

In [10]:
# once you compute you get a real pandas series
# type(unique_origins)

In [11]:
# unique_origins

In [34]:
# 2400 is not a valid time
df['CRSDepTime'] = df.apply(lambda row: 2359 if row['CRSDepTime'] == 2400 else row['CRSDepTime'], axis='columns')

In [13]:
# df.apply?

In [35]:
head = df.head()

[########################################] | 100% Completed | 19.5s


In [36]:
def create_timestamp (row):
    return pd.Timestamp('%s-%s-%s;%04d'%(row['Year'], row['Month'], row['DayofMonth'], row['CRSDepTime']))

In [16]:
# type(head)

In [17]:
# head

In [37]:
# create a sample for dask to figure out the data types
transformation_sample = head.apply(create_timestamp, axis='columns')

In [38]:
type(transformation_sample)

pandas.core.series.Series

In [39]:
transformation_sample[0]

Timestamp('2000-01-28 16:47:00')

In [40]:
# meta_information = {'@timestamp': pd.Timestamp}
meta_information = transformation_sample

df['@timestamp'] = df.apply(lambda row: pd.Timestamp('%s-%s-%s;%04d'%(row['Year'], row['Month'], row['DayofMonth'], row['CRSDepTime'])),
                            axis='columns',
                           meta=meta_information)

In [22]:
# df.head()

In [41]:
from pyelasticsearch import ElasticSearch, bulk_chunks

In [42]:
ES_HOST = 'http://localhost:9200/'
INDEX_NAME = "expo2009"
DOC_TYPE = "flight"
es = ElasticSearch(ES_HOST)

In [43]:
# https://pyelasticsearch.readthedocs.io/en/latest/api/#pyelasticsearch.ElasticSearch.bulk
def documents(records):
    for flight in records:
        yield es.index_op(flight)
        
def chunk_import(records):        
    # bulk_chunks() breaks your documents into smaller requests for speed:
    for chunk in bulk_chunks(documents(records=records),
                             docs_per_chunk=50000,
                             bytes_per_chunk=10000000):
        # We specify a default index and doc type here so we don't
        # have to repeat them in every operation:
        es.bulk(chunk, doc_type=DOC_TYPE, index=INDEX_NAME)

In [44]:
# should be 2 initially or 0, depending on if kibana hasrun already
es.count('*')['count']

2

In [45]:
df.npartitions

28

In [46]:
begin_partition = 0
end_partition = df.npartitions

# begin_partition = 23
# end_partition = 25

In [48]:
for partition_nr in range(df.npartitions):
    if partition_nr >= end_partition:
        break
    if partition_nr < begin_partition:
        continue
    print ("Importing partition %d"%(partition_nr))
    partition = df.get_partition(partition_nr)
    records = partition.compute().to_dict(orient='records')
    print ("Importing into ES: %d"%(len(records)))    
    chunk_import(records)
    cnt = es.count('*')['count']
    print ("Datasets in ES: %d"%(cnt))

Importing partition 0
[########################################] | 100% Completed |  2min 32.5s
Importing into ES: 640243
Datasets in ES: 638527
[                                        ] | 0% Completed |  0.0s
Importing partition 1
[########################################] | 100% Completed |  2min 34.6s
Importing into ES: 638869
Datasets in ES: 1276900
Time loading: 296.285
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  2min 34.0s
Importing into ES: 640046
Datasets in ES: 1913166
Time loading: 296.251
Importing partition 3
[########################################] | 100% Completed |  2min 34.4s
Importing into ES: 639869
Datasets in ES: 2552252
Time loading: 293.102
Importing partition 4
[########################################] | 100% Completed |  2min 34.2s
Importing into ES: 639036
Datasets in ES: 3197627
Time loading: 293.786
Importing partition 5
[########################################] | 100% C

In [ ]:
# for live reload of data during demo
# execute this and repeat steps from dd.read_csv in Cell 8
cd ..
mkdir feed
cd feed
!curl -O http://stat-computing.org/dataexpo/2009/2003.csv.bz2
!bzip2 -d 2003.csv.bz2